# Iris dataset

2 attributes

### Packages
- `pip install tensorflow==2.12`
- The current tensorflow version is 2.15 (as of Feb 2024).  However, We will use another package `scikeras` which requires older TF version.

In [ ]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

In [ ]:
# Read data
iris = datasets.load_iris()

# Create a DataFrame
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["target"] = iris.target

In [ ]:
# Shuffle the DataFrame (frac=1 means all rows)
# This is because the validatio split is done by taking the last % of the data. 
# If the data is not shuffled, the validation set will contain only the last class
print(df["target"].values)
df = df.sample(frac=1, random_state=0)
print(df.target.values)

In [ ]:
# 2 Attributes
X = df.iloc[:, 2:4].values
y = df["target"].values


# Split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Model building


In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(2,)),
        tf.keras.layers.Dense(units=16, activation="sigmoid", name="hidden1"),
        tf.keras.layers.Dense(units=3, activation="softmax", name="output"),
    ]
)

model.summary()

adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=adam, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Model training


In [ ]:
tf_fit = model.fit(
    x=X_train_std, y=y_train, batch_size=16, validation_split=0.2, epochs=200, verbose=1
)

In [ ]:
# Inspect the training history
df = pd.DataFrame(tf_fit.history)
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

df[["loss", "val_loss"]].plot(
    ax=axes[0], title="Loss", grid=True, legend=True, xlabel="Epoch", ylabel="Loss"
)

df[["accuracy", "val_accuracy"]].plot(
    ax=axes[1], title="Loss", grid=True, legend=True, xlabel="Epoch", ylabel="Loss"
)

## Model evaluation


In [ ]:
results = model.evaluate(X_test_std, y_test, verbose=0)
print(f"Test loss: {results[0]:.4f}   Test Acc.: {results[1]:.4f}")

## Prediction


In [ ]:
y_hat = model(X_test_std[:3])
print(y_hat.numpy())

In [ ]:
yp = y_hat.numpy()[0]
xp = list(range(yp.shape[0]))
plt.bar(xp, yp)
plt.title("Softmax Values")
plt.show()

In [ ]:
np.argmax(y_hat, axis=1)

## Decision surface


In [ ]:
XP = sc.transform(X)
x_min = XP[:, 0].min() - 0.1
x_max = XP[:, 0].max() + 0.1
y_min = XP[:, 1].min() - 0.1
y_max = XP[:, 1].max() + 0.1

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
f, ax = plt.subplots(1, 1, figsize=(6, 5))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = np.argmax(Z, axis=1)  # Find the class label
Z = Z.reshape(xx.shape)

ax.contourf(xx, yy, Z, alpha=0.3)
markers = ("s", "^", "o", "v", "x")
for idx, lb in enumerate(np.unique(y_test)):
    ax.scatter(
        X_test_std[y_test == lb, 0],
        X_test_std[y_test == lb, 1],
        marker=markers[idx],
        s=50,
    )

ax.set_title("ANN")
ax.set_xlabel("X1")
ax.set_ylabel("X2")
plt.tight_layout(pad=3.0)
plt.show()